In [2]:
!unzip Mask_noMask.zip

Archive:  Mask_noMask.zip
   creating: dataset/
   creating: dataset/with_mask/
  inflating: dataset/with_mask/0-with-mask.jpg  
   creating: __MACOSX/
   creating: __MACOSX/dataset/
   creating: __MACOSX/dataset/with_mask/
  inflating: __MACOSX/dataset/with_mask/._0-with-mask.jpg  
  inflating: dataset/with_mask/1-with-mask.jpg  
  inflating: __MACOSX/dataset/with_mask/._1-with-mask.jpg  
  inflating: dataset/with_mask/10-with-mask.jpg  
  inflating: __MACOSX/dataset/with_mask/._10-with-mask.jpg  
  inflating: dataset/with_mask/100-with-mask.jpg  
  inflating: __MACOSX/dataset/with_mask/._100-with-mask.jpg  
  inflating: dataset/with_mask/101-with-mask.jpg  
  inflating: __MACOSX/dataset/with_mask/._101-with-mask.jpg  
  inflating: dataset/with_mask/103-with-mask.jpg  
  inflating: __MACOSX/dataset/with_mask/._103-with-mask.jpg  
  inflating: dataset/with_mask/104-with-mask.jpg  
  inflating: __MACOSX/dataset/with_mask/._104-with-mask.jpg  
  inflating: dataset/with_mask/105-with-mask

In [30]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing.image import img_to_array,load_img,ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import  train_test_split 
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout ,Input
from tensorflow.keras.applications import imagenet_utils
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import cv2
import progressbar
import os

In [12]:
imagePaths=[]

In [13]:
for i in os.listdir("./dataset/"):
  for image in os.listdir("./dataset/" + i + "/"):
    path= "./dataset/" + i + "/" + image
    imagePaths.append(path)

In [14]:
imagePaths

['./dataset/without_mask/67.jpg',
 './dataset/without_mask/53.jpg',
 './dataset/without_mask/229.jpg',
 './dataset/without_mask/augmented_image_130.jpg',
 './dataset/without_mask/388.jpg',
 './dataset/without_mask/359.jpg',
 './dataset/without_mask/231.jpg',
 './dataset/without_mask/39.jpg',
 './dataset/without_mask/322.jpg',
 './dataset/without_mask/341.jpg',
 './dataset/without_mask/augmented_image_78.jpg',
 './dataset/without_mask/augmented_image_76.jpg',
 './dataset/without_mask/50.jpg',
 './dataset/without_mask/332.jpg',
 './dataset/without_mask/augmented_image_42.jpg',
 './dataset/without_mask/172.jpg',
 './dataset/without_mask/augmented_image_278.jpg',
 './dataset/without_mask/augmented_image_163.jpg',
 './dataset/without_mask/augmented_image_292.jpg',
 './dataset/without_mask/198.jpg',
 './dataset/without_mask/266.jpg',
 './dataset/without_mask/augmented_image_99.jpg',
 './dataset/without_mask/302.jpg',
 './dataset/without_mask/20.jpg',
 './dataset/without_mask/328.jpg',
 './da

In [18]:
data = []
labels = []

In [19]:
for i in imagePaths:
  image = load_img(i , target_size=(224,224))
  image = img_to_array(image)
  image = imagenet_utils.preprocess_input(image)
  

  label = i.split("/")[-2]

  data.append(image)
  labels.append(label)


In [24]:
len(data)

1376

In [25]:
len(labels)

1376

In [23]:
data =np.array(data)
labels = np.array(labels)

In [26]:
lb = LabelBinarizer()
Labels = lb.fit_transform(labels)

In [27]:
Labels

array([[1],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]])

In [28]:
Labels = to_categorical(Labels)

In [29]:
Labels

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [31]:
augment = ImageDataGenerator(rotation_range=30, height_shift_range=0.1, zoom_range=0.1 ,horizontal_flip=True, fill_mode="nearest", width_shift_range=0.1)

In [37]:
basemodel = VGG19(include_top=False , input_tensor=Input(shape=(224,224,3)))

In [38]:
headmodel = basemodel.output

In [39]:
headmodel = Flatten(name="flatten" )(headmodel)
headmodel= Dense(256 , activation="relu")(headmodel)
headmodel = Dropout(0.5)(headmodel)

headmodel = Dense(2 , activation="softmax" )(headmodel)

model = Model(inputs=basemodel.input , outputs = headmodel)

In [40]:
for layer in basemodel.layers:
  layer.trainable = False

In [41]:
model.summary()

Model: "functional_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [45]:
trainX , testX , trainY , testY  = train_test_split(data , Labels , test_size=0.2)

In [43]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])


In [46]:
print("Model is going to Train.......")
H = model.fit(augment.flow(trainX, trainY, batch_size=32), validation_data=(testX, testY), epochs=7)

Model is going to Train.......
Epoch 1/7
35/35 [==============================] - 51s 1s/step - loss: 3.6763 - accuracy: 0.9318 - val_loss: 0.0145 - val_accuracy: 0.9964
Epoch 2/7
35/35 [==============================] - 50s 1s/step - loss: 0.3945 - accuracy: 0.9918 - val_loss: 0.1555 - val_accuracy: 0.9964
Epoch 3/7
35/35 [==============================] - 50s 1s/step - loss: 0.2204 - accuracy: 0.9936 - val_loss: 0.3297 - val_accuracy: 0.9964
Epoch 4/7
35/35 [==============================] - 50s 1s/step - loss: 0.3163 - accuracy: 0.9918 - val_loss: 0.1763 - val_accuracy: 0.9964
Epoch 5/7
35/35 [==============================] - 51s 1s/step - loss: 0.7793 - accuracy: 0.9918 - val_loss: 0.5538 - val_accuracy: 0.9928
Epoch 6/7
35/35 [==============================] - 53s 2s/step - loss: 0.2531 - accuracy: 0.9945 - val_loss: 0.9207 - val_accuracy: 0.9928
Epoch 7/7
35/35 [==============================] - 49s 1s/step - loss: 0.2374 - accuracy: 0.9936 - val_loss: 0.6820 - val_accuracy: 0.9

In [47]:
model.save("mask_detection.h5")